# Demo 10: Time-Affinity Optimization

This notebook demonstrates walltime-based parameter discovery and optimization.

**Key Concepts:**
- Use execution time as a fitness signal
- Discover optimal parameters empirically
- Diagnostic tool for unknown relationships
- Correct parameters → Less work → Faster execution

**Use Case**: When you don't know the optimal parameter values but suspect that correct values will make the algorithm run faster.

Run all cells to see outputs.

In [ ]:
import numpy as np
import time
from workbench import (
    TimeAffinityOptimizer,
    GridSearchTimeAffinity,
    quick_calibrate,
    TimeAffinityResult
)

print('=' * 70)
print('DEMO 10: TIME-AFFINITY OPTIMIZATION')
print('=' * 70)

## Part 1: Basic Concept - Finding Optimal Complexity

Suppose we have an algorithm with unknown optimal parameters. We can use time-affinity to discover them.

In [ ]:
# Algorithm with hidden optimal parameters
def mystery_algorithm(threshold, iterations):
    """
    This algorithm has a 'sweet spot' where it runs fastest.
    Optimal: threshold ≈ 0.5, iterations ≈ 10
    """
    result = 0
    
    # Inefficient if threshold too high or too low
    penalty = abs(threshold - 0.5) * 1000
    
    # Inefficient if iterations far from optimal
    penalty += abs(iterations - 10) * 50
    
    for i in range(int(100 + penalty)):
        result += np.sum(np.random.rand(50))
    
    return result

print('Mystery Algorithm: Unknown optimal parameters')
print('Goal: Discover parameters that minimize execution time')
print()

# Test a few random configurations
print('Testing random configurations:')
for threshold, iterations in [(0.1, 5), (0.5, 10), (0.9, 20)]:
    start = time.perf_counter()
    mystery_algorithm(threshold, iterations)
    elapsed = time.perf_counter() - start
    print(f'  threshold={threshold:.1f}, iterations={iterations:2d}: {elapsed:.6f}s')

In [ ]:
# Use time-affinity to discover optimal parameters
print('\nUsing Time-Affinity to discover optimal parameters:')
print('-' * 70)

# We'll target a fast time (e.g., 0.01s) and let the optimizer find params
result = quick_calibrate(
    mystery_algorithm,
    target_time=0.01,
    param_bounds={
        'threshold': (0.0, 1.0),
        'iterations': (1.0, 30.0)
    },
    method='gradient',
    max_iterations=30,
    verbose=False
)

print(f'\nDiscovered optimal parameters:')
print(f'  threshold:  {result.best_params["threshold"]:.3f} (true optimal: 0.500)')
print(f'  iterations: {result.best_params["iterations"]:.1f} (true optimal: 10.0)')
print(f'\nAchieved time: {result.best_time:.6f}s')
print(f'Iterations: {result.iterations}')

## Part 2: Diagnostic Use - Discovering Relationships

Time-affinity can reveal unknown parameter relationships by observing which combinations run fastest.

In [ ]:
# Algorithm with hidden relationship: optimal when x * y ≈ 10
def coupled_algorithm(x, y):
    """
    Hidden relationship: x * y should equal 10 for optimal performance
    """
    product = x * y
    penalty = abs(product - 10) * 100
    
    for i in range(int(100 + penalty)):
        _ = np.sum(np.random.rand(30))
    
    return product

print('Coupled Algorithm: Parameters have hidden relationship')
print('Goal: Discover the relationship x * y = 10')
print()

# Use grid search to map the parameter space
optimizer = GridSearchTimeAffinity(
    target_time=0.005,
    param_grids={
        'x': np.linspace(1, 20, 10),
        'y': np.linspace(0.5, 10, 10)
    },
    warmup_runs=2
)

result = optimizer.optimize(coupled_algorithm, verbose=False)

print(f'Discovered parameters:')
print(f'  x = {result.best_params["x"]:.2f}')
print(f'  y = {result.best_params["y"]:.2f}')
print(f'  x * y = {result.best_params["x"] * result.best_params["y"]:.2f} (target: 10.0)')
print(f'\nTime: {result.best_time:.6f}s')
print(f'\n✓ Discovered the hidden relationship empirically!')

## Part 3: Real-World Example - SRT Parameter Discovery

Discover optimal SRT-style parameters for a spectral algorithm.

In [ ]:
from workbench import SpectralScorer, ZetaFiducials

# Create a spectral scoring task
zeros = ZetaFiducials.get_standard(20)
candidates = np.arange(1000, 2000)

def spectral_task(damping, shift):
    """
    Spectral scoring with variable parameters.
    Optimal parameters should minimize computation.
    """
    scorer = SpectralScorer(frequencies=zeros, damping=damping)
    scores = scorer.compute_scores(candidates, shift=shift, mode='real')
    return scores

print('Real-World: Discovering optimal spectral parameters')
print('-' * 70)

# Find parameters that make spectral scoring run in ~0.05s
result = quick_calibrate(
    spectral_task,
    target_time=0.05,
    param_bounds={
        'damping': (0.01, 0.2),
        'shift': (0.0, 0.1)
    },
    method='gradient',
    max_iterations=20,
    verbose=False
)

print(f'Discovered parameters:')
print(f'  damping: {result.best_params["damping"]:.4f}')
print(f'  shift:   {result.best_params["shift"]:.4f}')
print(f'\nExecution time: {result.best_time:.6f}s')
print(f'Target time:    {result.target_time:.6f}s')
print(f'Error:          {result.time_error:.6f}s')

## Part 4: Convergence Analysis

In [ ]:
# Analyze convergence behavior
def analysis_algorithm(alpha, beta):
    penalty = (alpha - 0.3)**2 + (beta - 0.7)**2
    for i in range(int(50 + penalty * 1000)):
        _ = np.sum(np.random.rand(20))
    return alpha + beta

print('Convergence Analysis:')
print('-' * 70)

optimizer = TimeAffinityOptimizer(
    target_time=0.003,
    param_bounds={'alpha': (0.0, 1.0), 'beta': (0.0, 1.0)},
    max_iterations=30,
    learning_rate=0.1,
    momentum=0.5
)

result = optimizer.optimize(analysis_algorithm, verbose=False)

print(f'\nFinal parameters:')
print(f'  alpha: {result.best_params["alpha"]:.3f} (optimal: 0.300)')
print(f'  beta:  {result.best_params["beta"]:.3f} (optimal: 0.700)')

# Show convergence
print(f'\nTime evolution (first 10 iterations):')
for i, t in enumerate(result.time_history[:10]):
    error = abs(t - result.target_time)
    print(f'  Iter {i:2d}: time={t:.6f}s, error={error:.6f}s')

print(f'\nConvergence rate: {result.convergence_rate:.6f}s/iter')

## Part 5: Grid Search vs Gradient Descent

In [ ]:
# Compare methods
def test_algorithm(p1, p2):
    penalty = (p1 - 0.6)**2 + (p2 - 0.4)**2
    for i in range(int(30 + penalty * 500)):
        _ = np.sum(np.random.rand(15))
    return p1 * p2

print('Method Comparison: Grid Search vs Gradient Descent')
print('=' * 70)

# Gradient descent
print('\n1. Gradient Descent:')
start = time.time()
result_grad = quick_calibrate(
    test_algorithm,
    target_time=0.002,
    param_bounds={'p1': (0.0, 1.0), 'p2': (0.0, 1.0)},
    method='gradient',
    max_iterations=20,
    verbose=False
)
time_grad = time.time() - start

print(f'   Found: p1={result_grad.best_params["p1"]:.3f}, p2={result_grad.best_params["p2"]:.3f}')
print(f'   Time error: {result_grad.time_error:.6f}s')
print(f'   Optimization time: {time_grad:.3f}s')

# Grid search
print('\n2. Grid Search:')
start = time.time()
result_grid = quick_calibrate(
    test_algorithm,
    target_time=0.002,
    param_bounds={'p1': (0.0, 1.0), 'p2': (0.0, 1.0)},
    method='grid',
    grid_points=10,
    verbose=False
)
time_grid = time.time() - start

print(f'   Found: p1={result_grid.best_params["p1"]:.3f}, p2={result_grid.best_params["p2"]:.3f}')
print(f'   Time error: {result_grid.time_error:.6f}s')
print(f'   Optimization time: {time_grid:.3f}s')

print('\nComparison:')
print(f'   Gradient: {result_grad.iterations} evaluations in {time_grad:.3f}s')
print(f'   Grid:     {result_grid.iterations} evaluations in {time_grid:.3f}s')
print(f'   Speedup:  {time_grid/time_grad:.1f}x faster with gradient descent')

## Part 6: Diagnostic Use Case - Unknown Algorithm Behavior

In [ ]:
# Simulate discovering unknown algorithm behavior
def black_box_algorithm(param_a, param_b, param_c):
    """
    Complex algorithm with unknown optimal configuration.
    Hidden rule: param_a + 2*param_b - param_c should equal 5
    """
    target_value = 5
    actual_value = param_a + 2*param_b - param_c
    penalty = abs(actual_value - target_value) * 200
    
    for i in range(int(50 + penalty)):
        _ = np.sum(np.random.rand(25))
    
    return actual_value

print('Diagnostic Use Case: Discovering Unknown Relationships')
print('=' * 70)
print('\nScenario: Black-box algorithm with unknown optimal parameters')
print('Goal: Discover parameter values empirically using time as signal')
print()

# Use time-affinity to discover the relationship
result = quick_calibrate(
    black_box_algorithm,
    target_time=0.003,
    param_bounds={
        'param_a': (0.0, 10.0),
        'param_b': (0.0, 10.0),
        'param_c': (0.0, 10.0)
    },
    method='gradient',
    max_iterations=40,
    verbose=False
)

a = result.best_params['param_a']
b = result.best_params['param_b']
c = result.best_params['param_c']

print(f'Discovered parameters:')
print(f'  param_a = {a:.2f}')
print(f'  param_b = {b:.2f}')
print(f'  param_c = {c:.2f}')
print(f'\nDiscovered relationship:')
print(f'  param_a + 2*param_b - param_c = {a + 2*b - c:.2f}')
print(f'  (Hidden target value: 5.0)')
print(f'\n✓ Successfully discovered the hidden constraint empirically!')

## Summary

**Time-Affinity Optimization** is a diagnostic tool for:

**Primary Use Cases:**
1. **Parameter Discovery**: Find optimal values when you don't know them
2. **Relationship Discovery**: Reveal hidden parameter relationships
3. **Empirical Calibration**: Tune algorithms without theoretical models
4. **Performance Diagnosis**: Understand why certain configs run faster

**Key Principle:**
- Correct/resonant parameters → Algorithm does less work → Faster execution
- Walltime becomes a proxy for parameter quality

**When to Use:**
- Unknown optimal parameter values
- Suspected parameter relationships
- Black-box algorithm tuning
- Empirical performance analysis

**Methods:**
- **Gradient Descent**: Fast, good for continuous spaces
- **Grid Search**: Exhaustive, good for discrete/complex spaces

**Limitations:**
- Requires algorithm to have time-parameter relationship
- Timing noise can affect convergence
- Local minima possible with gradient descent

**Best Practices:**
- Use multiple warmup runs to reduce noise
- Start with grid search for exploration
- Refine with gradient descent
- Verify discovered parameters make sense